# setting up cuda C++ in colab and installing cantera
use %%cu to deliniate C++ code at the begining

In [0]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!nvcc --version

!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-410' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-415' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-418' for glob 'nvidia*'
Note, selecting 'nvidi

In [0]:
!sudo apt install python-software-properties 
!sudo apt-add-repository ppa:speth/cantera 
!sudo apt update 
!sudo apt install cantera-python cantera-python3 cantera-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package python-software-properties is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  software-properties-common

E: Package 'python-software-properties' has no installation candidate
 Ubuntu packages for Cantera, an object-oriented software toolkit for chemical kinetics, thermodynamics, and transport processes, which can be used with Python, C++ and Fortran 90.

Matlab packages are not available from this PPA.
 More info: https://launchpad.net/~speth/+archive/ubuntu/cantera
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 file:/var/cuda-repo-9-2-local  InRelease
Ign:1 file:/var/cuda-repo-9-2-local  InRelease
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Hit:3 https

#load functions

In [0]:
import cantera as ct 
import numpy as np 
import requests
import urllib.request
import re
from bs4 import BeautifulSoup

#https://www.grc.nasa.gov/www/k-12/airplane/specimp.html

def inputs(Desired_thrust,Desired_Spesific_Impulse_SL,Fuel_Oxidiser,Equivilence_Ratio):

    mass_flow_rate = Desired_thrust/(Desired_Spesific_Impulse_SL*9.80665)

    if Fuel_Oxidiser =="H2 + Air":
      stoichiometric_fuel_to_ox = 0.029285
      ch4o2 = 0
      h2air = 1
    elif Fuel_Oxidiser == "CH4 + O2":
      stoichiometric_fuel_to_ox = 0.250625
      ch4o2 = 1
      h2air = 0
    else: 
      print(f'{Fuel_Oxidiser} is an invalid input,try either "H2 + Air" or "CH4 + O2".')

    mf_mox_ratio = Equivilence_Ratio*stoichiometric_fuel_to_ox

    m_fu = mass_flow_rate/ (1 + (1/mf_mox_ratio))
    m_ox = m_fu * (1/mf_mox_ratio)

    return m_fu,m_ox

#---------------------------------------------------------------------------------------------------------------------------

#http://dx.doi.org/10.1016/j.actaastro.2015.02.006

def injector(d_minor_fu,d_minor_ox,k,beta,alpha,length): #angles in deg, lengths in mm
  k = k
  D_major_fu = d_minor_fu / np.cos(np.deg2rad(alpha))
  D_major_ox = d_minor_ox / np.cos(np.deg2rad(alpha))

  a_fu = D_major_fu / 2
  b_fu = d_minor_fu / 2

  a_ox = D_major_ox / 2
  b_ox = d_minor_ox / 2

  r_at_alpha_fu = a_fu*b_fu / np.sqrt(((b_fu*np.cos(np.deg2rad(alpha)))**2)+((a_fu*np.sin(np.deg2rad(alpha)))**2))
  r_at_alpha_ox = a_ox*b_ox / np.sqrt(((b_ox*np.cos(np.deg2rad(alpha)))**2)+((a_ox*np.sin(np.deg2rad(alpha)))**2))

  x_len_of_r_fu = r_at_alpha_fu*np.cos(np.deg2rad(beta))
  x_len_of_r_ox = r_at_alpha_ox*np.cos(np.deg2rad(beta))

  y_len_of_r_fu =  r_at_alpha_fu*np.sin(np.deg2rad(beta))
  y_len_of_r_ox = r_at_alpha_ox*np.sin(np.deg2rad(beta))

  a_box = a_fu + a_ox + x_len_of_r_fu + x_len_of_r_ox
  b_box = b_fu + b_ox + y_len_of_r_fu + y_len_of_r_ox

  a_outer_box = a_box + 2*k
  b_outer_box = b_box + 2*k

  difference_a = (a_outer_box - a_box)/2
  difference_b = (a_outer_box - a_box)/2
  
  fuel_centerpoint_x = a_fu
  fuel_centerpoint_y = b_box - b_fu

  ox_centerpoint_x =  a_box - a_ox
  ox_centerpoint_y =  b_ox

  return d_minor_fu,d_minor_ox,D_major_fu,D_major_ox,a_box,b_box,a_outer_box,b_outer_box,fuel_centerpoint_x,fuel_centerpoint_y,ox_centerpoint_x,ox_centerpoint_y,beta,alpha,length,k

#---------------------------------------------------------------------------------------------------------------------------

# pressure is in atm

def inputs_2(Desired_thrust,Desired_Spesific_Impulse_SL,Fuel_Oxidiser,Equivilence_Ratio,Inital_Temperature,Inital_Pressure):

   #https://www.grc.nasa.gov/www/k-12/airplane/specimp.html

    mass_flow_rate = Desired_thrust/(Desired_Spesific_Impulse_SL*9.80665)

    if Fuel_Oxidiser =="H2 + Air":
      stoichiometric_fuel_to_ox = 0.029285
      ch4o2 = 0
      h2air = 1
    elif Fuel_Oxidiser == "CH4 + O2":
      stoichiometric_fuel_to_ox = 0.250625
      ch4o2 = 1
      h2air = 0
    else: 
      print(f'{Fuel_Oxidiser} is an invalid input,try either "H2 + Air" or "CH4 + O2".')

    mf_mox_ratio = Equivilence_Ratio*stoichiometric_fuel_to_ox

    m_fu = mass_flow_rate/ (1 + (1/mf_mox_ratio))
    m_ox = m_fu * (1/mf_mox_ratio)

    #Mass fraction

    fuel_mass_fraction = m_fu / mass_flow_rate

    #Adiabatic Flame Temperature 
    # https://cantera.org/examples/python/multiphase/adiabatic.py.html
    T = Inital_Temperature
    P = Inital_Pressure*101325
    gas = ct.Solution('gri30.xml')
    carbon = ct.Solution('graphite.xml')
    mix_phases = [(gas, 1.0), (carbon, 0.0)]
    
    if Fuel_Oxidiser =="H2 + Air":
        # gaseous fuel species
        fuel_species = 'H2'

        mix = ct.Mixture(mix_phases)

        # set the gas state
        gas.set_equivalence_ratio(Equivilence_Ratio, fuel_species, 'O2:1.0, N2:3.76')

        # create a mixture of 1 mole of gas, and 0 moles of solid carbon.
        mix = ct.Mixture(mix_phases)
        mix.T = T
        mix.P = P

        # equilibrate the mixture adiabatically at constant P
        mix.equilibrate('HP', solver='gibbs', max_steps=1000)
        AFT = mix.T
    else: 
        fuel_species = 'CH4'

        mix = ct.Mixture(mix_phases)

        # set the gas state
        gas.set_equivalence_ratio(Equivilence_Ratio, fuel_species, 'O2:2.0')

        # create a mixture of 1 mole of gas, and 0 moles of solid carbon.
        mix = ct.Mixture(mix_phases)
        mix.T = T
        mix.P = P

        # equilibrate the mixture adiabatically at constant P
        mix.equilibrate('HP', solver='gibbs', max_steps=1000)
        AFT = mix.T

    return fuel_mass_fraction,AFT,Inital_Pressure,ch4o2,h2air

#---------------------------------------------------------------------------------------------------------------------------

# mm = output , n = no. of detonation waves

#DOI: 10.2514/1.17656 , Bykovskii et Al.

def geo(mm,n):
  if n % 1 == 0:
    length_opt_min = 28*mm
    length_opt_max = 68*mm
    Dc_min = 14*n*mm
    Dc_max = 34*n*mm
  else: 
    print("n must be a whole number")
  return length_opt_min,length_opt_max,Dc_min,Dc_max


#---------------------------------------------------------------------------------------------------------------------------

# H2/air, pressure and temperature to get density function, https://www.gribble.org/cycling/air_density.html
# Pressure in Pa, temp in Celsius
def density_air(Pressure,Temp,Temp_dewpoint):
  
  Pressure = Pressure / 100
  
  e_so = 6.1078
  c0 = 0.99999683
  c1 = -0.90826951 * 10**-2
  c2 = 0.78736169 * 10**-4
  c3 = -0.61117958 * 10**-6
  c4 = 0.43884187 * 10**-8
  c5 = -0.29883885 * 10**-10
  c6 = 0.21874425 * 10**-12
  c7 = -0.17892321 * 10**-14
  c8 = 0.11112018 * 10**-16
  c9 = -0.30994571 * 10**-19
  Rv = 461.4964
  Rd = 287.0531
  
  p = c0 + Temp_dewpoint*(c1 + Temp_dewpoint*(c2 + Temp_dewpoint*(c3 + Temp_dewpoint*(c4 + Temp_dewpoint*(c5 + Temp_dewpoint*(c6 + Temp_dewpoint*(c7 + Temp_dewpoint*(c8 + Temp_dewpoint*(c9)))))))))
  
  Pv = e_so / p**8
  
  Pd = Pressure - Pv
  
  Tk = Temp + 273.15
  
  density_of_air = ((Pd / (Rd * Tk)) + (Pv / (Rv * Tk))) * 100
  
  return density_of_air

#---------------------------------------------------------------------------------------------------------------------------
# H2/air, pressure and temperature to get density function
# Pressure in Pa, temp in Celsius
def density_h2(Pressure, Temp):
  P = Pressure / 1000000
  T = Temp + 273.15
  p_url = str(P)
  t_url = str(T)
  t_maxval = T + 10
  t_max_url = str(t_maxval)
  
  url =  "https://webbook.nist.gov/cgi/fluid.cgi?Action=Load&ID=C1333740&Type=IsoBar&Digits=5&P=" + p_url + "&THigh=" + t_max_url + "&TLow=" + t_url + "&TInc=10&RefState=DEF&TUnit=K&PUnit=MPa&DUnit=kg%2Fm3&HUnit=kJ%2Fmol&WUnit=m%2Fs&VisUnit=Pa*s&STUnit=N%2Fm"

  response = requests.get(url)

  soup = BeautifulSoup(response.text, "html.parser")

  found = soup.findAll('td')

  found2 = re.findall('\d*\.?\d+',str(found[2]))

  found3 = found2[0].split('.')
  pt1 = int(found3[0])
  pt2 = int(found3[1])
  length = (int(len(found3[1])))
  pt2_decimal = pt2 / (10**length)
  density = pt1 + pt2_decimal
  
  return density

#---------------------------------------------------------------------------------------------------------------------------

# ch4/o2, pressure and temperature to get density function
# Pressure in Pa, temp in Celsius

def density_o2(Pressure, Temp):
  P = Pressure / 1000000
  T = Temp + 273.15
  p_url = str(P)
  t_url = str(T)
  t_maxval = T + 10
  t_max_url = str(t_maxval)
  
  url =  "https://webbook.nist.gov/cgi/fluid.cgi?Action=Load&ID=C7782447&Type=IsoBar&Digits=5&P=" + p_url + "&THigh=" + t_max_url + "&TLow=" + t_url + "&TInc=10&RefState=DEF&TUnit=K&PUnit=MPa&DUnit=kg%2Fm3&HUnit=kJ%2Fmol&WUnit=m%2Fs&VisUnit=Pa*s&STUnit=N%2Fm"
  response = requests.get(url)
  
  soup = BeautifulSoup(response.text, "html.parser")

  found = soup.findAll('td')

  found2 = re.findall('\d*\.?\d+',str(found[2]))

  found3 = found2[0].split('.')
  pt1 = int(found3[0])
  pt2 = int(found3[1])
  length = (int(len(found3[1])))
  pt2_decimal = pt2 / (10**length)
  density = pt1 + pt2_decimal
  
  return density

#---------------------------------------------------------------------------------------------------------------------------

# ch4/o2, pressure and temperature to get density function
# Pressure in Pa, temp in Celsius

def density_ch4(Pressure, Temp):
  P = Pressure / 1000000
  T = Temp + 273.15
  p_url = str(P)
  t_url = str(T)
  t_maxval = T + 10
  t_max_url = str(t_maxval)
  
  url =  "https://webbook.nist.gov/cgi/fluid.cgi?Action=Load&ID=C74828&Type=IsoBar&Digits=5&P=" + p_url + "&THigh=" + t_max_url + "&TLow=" + t_url + "&TInc=10&RefState=DEF&TUnit=K&PUnit=MPa&DUnit=kg%2Fm3&HUnit=kJ%2Fmol&WUnit=m%2Fs&VisUnit=Pa*s&STUnit=N%2Fm"
 
  response = requests.get(url)

  soup = BeautifulSoup(response.text, "html.parser")

  found = soup.findAll('td')

  found2 = re.findall('\d*\.?\d+',str(found[2]))

  found3 = found2[0].split('.')
  pt1 = int(found3[0])
  pt2 = int(found3[1])
  length = (int(len(found3[1])))
  pt2_decimal = pt2 / (10**length)
  density = pt1 + pt2_decimal
  
  return density

#---------------------------------------------------------------------------------------------------------------------------


#run functions

##diagrams of simulated injector geometry

![alt text](https://lh3.googleusercontent.com/gwDfYc_ogcc-T-S0G98ER62nBu9Biffyfyvc6UvR-a2Y-86euyPZNDYLvSXLQa5zl8Qi2TC44jsGdWRPuL2QrjOtsy5Jf6P4qLsfq_LAxKIghRhjeDfXiuSe0hW0cMJefCMFqmJDSQ6gK8Mh2oD0kWCTtGb8F7DfGoM0xuCdNW1_bxivXGvjbGRPLKQPIfhen0hHOngPE8pxr9hcuTAD-m0VlfrfvYAbXh8DFe5dyerxFHiE7s1YYkfPlOdtxOlCeWTOqE_2U754hHVGKikmUzeGX3snA8R2OQ0WJhpKePfUgJlFwaLUhZurfAuF54pzMlNuxcDpqds40jQ1IfhX1D809vksdge_WOOqGOp0INUG7u_acMIzqnV5SO2ApVZb-gn8k2sG1Z8qeQTKvDFKWoFCJABb48w7fulQGOMrvHx-RpFYfmGxSZBnggdNYqkRZ1nQOmjzCfyn8UC09F00dqeznxw7h4C467KEp8hvIka0ne49qK5Gm0QFaKJhHNcTZnWt_IUiptVUdjjBivY1Gb6HPyRGUBnod91jWNlhP7MSkpuNhPaeC2MNC73VGGlZXVONEn88aqCo6b5wB2v8cn06lMZWf9SfA50DV-K1cXkT8snMod-oHHP72M4VlPTqP1EesEmgCWk97EfBz69AvfqPklbNKz3IPnB0bxAOytXoVXyIeLX4iTo=w902-h638-no)



![alt text](https://lh3.googleusercontent.com/6SXZn1W3ETk8_CDuoELzfVel3fE2NQk6D6-v3RnTwOHrq4Nfpv3NNcJJ6mm_xJWZ3xpfEVTwz0rAqQOucCw6hcbpP1uStwb0ItLH44HHRat5Bh0KScaZ9yq18yIP8_xVqbdD_dKxON1QfX9Wmzpm2xhUjzFwMVLsHOOc6VznfOaX4msXxo8kFSyaCDfOFE4Wmvs-u9Zsw1jrw_B1vPQKwCOc_4dP01gTNtVqkNGPz8T8KwzsV9UFtwkNS0ql3iVAj68hjQbpKwnSo3CwWzd0agEUO71TLHN8zTJWUuIxSx2OVlc0vGULvZx2V07uOv6Rdx6d6Tk7GXm_Xti93dN5FDhq-0-GbG3iFqrCZOhF2eG1kr3ISFC2WhSAdYMEcqcXSX7lRezYuZ8_DiKdOIfuQd1t33EHlOjS-0TwwIzKH4G2ZbG8XSRU57fgul50StQEGGQbAdu4K_EhNwBAq-3nK834Lc2N4rauD00VTm1f2fnNIEFR8pXDnDeIp83gVAK-ep4duJS7eG8QJdkJ-WS_cqHNnAVoGVdBoDP-P_9VZPL2hwJCemNEd4Te5zJOZEh-7K3muaJaHc6DlbpiqDWKjk00b4-oMBP-05tBB_8YT9bHbFroZQuK-Bhu73pl3R7gxrSFbvyXDHcoIRUyAZTPCTvlS_nspqoc2TLsk0_H_6Mbh9HM_cljp9E=w1181-h886-no)



## Running the injector simulation

In [0]:
###inputs
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd

"""edit hyperpameters to desired values

"""

desired_thrust = 1000000               # N
expected_intial_spesific_impulse = 7000# s
equivilence_ratio_evlauation = 1       # 1 = Hightest efficency
fuel_oxidiser ="CH4 + O2"              # "H2 + Air" or "CH4 + O2"
d_minor_fu = 2                         # mm
d_minor_ox = 4                         # mm
k = 0.3                                # mm
beta = 30                              # degrees
alpha = 30                             # degrees
length = 15                            # mm
pressure_air = 101325                  # Pa
temperature_air = 30                   # C
pressure_h2 = 101325                   # Pa
temperature_h2 = 30                    # C
pressure_o2 = 101325                   # Pa
temperature_o2 = 30                    # C
pressure_ch4 = 101325                  # Pa
temperature_ch4 = 30                   # C
temperature_dewpoint = 7.5             # C
no_of_injectors = 1000                 # no. of pairs of fu/ox injectors
n = 1                                  # no. of detonation waves


det_inputs = inputs(desired_thrust,expected_intial_spesific_impulse,fuel_oxidiser,equivilence_ratio_evlauation)

if fuel_oxidiser == "H2 + Air":

  density_ox = density_air(pressure_air,temperature_air,temperature_dewpoint)
  density_fu = density_h2(pressure_h2,temperature_h2)

else:

  density_ox = density_o2(pressure_o2,temperature_o2)
  density_fu = density_ch4(pressure_ch4,temperature_ch4)

flow_velocity_fuel = det_inputs[0] / (density_fu * no_of_injectors * ((np.pi * (d_minor_fu/1000)**2)/4))
flow_velocity_oxidiser = det_inputs[1] / (density_ox * ((np.pi * no_of_injectors * (d_minor_ox/1000)**2)/4))


det_injector = injector(d_minor_fu,d_minor_ox,k,beta,alpha,length)

list_1 = np.asarray(det_injector)
list_of = np.array(["d_minor_fu", "d_minor_ox", "D_major_fu", "D_major_ox", "a_box", "b_box","a_outer_box", "b_outer_box", "fuel_centerpoint_x", "fuel_centerpoint_y", "ox_centerpoint_x", "ox_centerpoint_y", "beta", "alpha","length","k"])
list_2 = np.array(["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16"])

injector_design = pd.DataFrame([list_1,list_2], columns = list_of)
display(injector_design)  
  
print(f"Fuel flow velocity is {flow_velocity_fuel} ms^-1, mass flow rate is {det_inputs[0]} kg/m^3 and density is {density_fu} kg/m^3")
print(f"Oxidiser flow velocity is {flow_velocity_oxidiser} ms^-1, mass flow rate is {det_inputs[1]} kg/m^3 and density is {density_ox} kg/m^3")

,d_minor_fu,d_minor_ox,D_major_fu,D_major_ox,a_box,b_box,a_outer_box,b_outer_box,fuel_centerpoint_x,fuel_centerpoint_y,ox_centerpoint_x,ox_centerpoint_y,beta,alpha,length,k
0,2,4,2.3094,4.6188,6.34641,4.6641,6.94641,5.2641,1.1547,3.6641,4.03701,2,30,30,15,0.3
1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16


Fuel flow velocity is 1438.499196227287 ms^-1, mass flow rate is 2.9192989319922122 kg/m^3 and density is 0.64598 kg/m^3
Oxidiser flow velocity is 720.1650219714508 ms^-1, mass flow rate is 11.648075539121047 kg/m^3 and density is 1.2871000000000001 kg/m^3


## using conditions from injector to calculate chamber diamentions

In [0]:
"""edit parameters to found values

"""

injector_outlet_pressure = 10           # atm
injector_outlet_temperature = 22        # K


det_ann_inputs = inputs_2(desired_thrust,expected_intial_spesific_impulse,fuel_oxidiser,equivilence_ratio_evlauation,injector_outlet_temperature,injector_outlet_pressure)

f= open("ann_inputs.txt","w+")
arrItems = str(det_ann_inputs)[1:-1].split(", ")
for item in arrItems:
  f.write(item + "\n")
f.close() 

## detonation cell neural network
https://doi.org/10.1016/j.net.2018.11.004

https://github.com/konradmalik/ann-detonation-cell-sizes/blob/master/ann-detonation-cells_vol_press.C

In [0]:
%%capture output
%%cu



#include <stdio.h>
#include <math.h>
#include <fstream>
#include <iostream>
using namespace std;





/* function declaration - network evaluation */
void export_eval(const double*, double*);


/* ------------------------------------------------------------ */


static const double export_in_A[25] = {
 6.006006006006007, 0, 0, 0,
 0, 0, 0.001417786622391395, 0,
 0, 0, 0, 0,
 2.192669904509226, 0, 0, 0,
 0, 0, 2, 0,
 0, 0, 0, 0,
 2
};


static const double export_in_b[5] = {
 -2.003003003003003, -3.332664654340537, -1.195410741889862, -1,
 -1
};


void
export_in(const double *x, double *y)
{
    int k = 0, i, j;
    for (i = 0; i < 5; ++i) {
        y[i] = export_in_b[i];
        for (j = 0; j < 5; ++j, ++k) {
            y[i] += export_in_A[k] * x[j];
        }
    }
}


static const double export_out_A[1] = {
 1.147315646043288
};


static const double export_out_b[1] = {
 1.358714706241739
};


void
export_out(const double *x, double *y)
{
    int k = 0, i, j;
    for (i = 0; i < 1; ++i) {
        y[i] = export_out_b[i];
        for (j = 0; j < 1; ++j, ++k) {
            y[i] += export_out_A[k] * x[j];
        }
    }
}


static const double export_w[59] = {
 0.2422731102424329, 0.1685601972805633, 1.144938635478179, 0.9251465781146723,
 -0.3401490114059418, 0.1849678968515714, -0.2109478060151187, -0.9994219108215934,
 0.5609672424685861, -0.5316586607126723, -0.0401498587176148, -0.000311970886502372,
 -0.8011945440936979, -1.315131443939578, -1.335786196344254, -1.734336818161753,
 0.6471954961139879, -0.6403800015658664, -0.5447646091843412, -0.4500650775295008,
 0.8424459987663341, 0.2467634469991534, -0.1100576163505895, 0.1279588219121385,
 -0.7596559067824056, 1.30605189935239, -0.5397504283725704, 1.449720237173661,
 -0.1692412128968907, 0.1735480456960122, -0.02415153822775668, -2.809354898439365,
 -0.8557470785050109, 0.400547923937851, -1.581420261971157, 1.860398142419893,
 -0.005549380475841786, 0.04622575406816191, 0.2690885808897128, -0.09123800297388644,
 0.9836935045580477, 0.4524265399665353, 0.0898692268235641, -0.7731193511271613,
 1.684743685007502, -0.7992532153341293, -0.6515485504497029, 0.1360072220929812,
 0.8958239310893573, 0.2777879546041405, -1.022831814586224, 0.8449967503232536,
 -2.289901661486862, 0.8292327421889349, -0.1064593336973993, 0.3629656315892187,
 0.4580302834404282, 0.8169688993126462, 0.3974509832838855
};


void
export_eval(const double *input, double *output)
{
    double n[15];
    double x;
    int k = 0, i = 5, j;

    export_in(input, n);
    for (; i < 10; ++i) {
        x = export_w[k++];
        for (j = 0; j < 5; ++j) x += n[j] * export_w[k++];
        n[i] = (double)2. / ((double)1. + exp((double)-2 * x)) - (double)1.;
    }
    for (; i < 14; ++i) {
        x = export_w[k++];
        for (j = 5; j < 10; ++j) x += n[j] * export_w[k++];
        n[i] = (double)2. / ((double)1. + exp((double)-2 * x)) - (double)1.;
    }
    for (; i < 15; ++i) {
        x = export_w[k++];
        for (j = 10; j < 14; ++j) x += n[j] * export_w[k++];
        n[i] = x;
    }
    export_out(n + 14, output);
}

int main()
{
    
    double testinp[7];  
   ifstream infile; 
   infile.open("/content/ann_inputs.txt");
string line;

int i = 0;
if (infile.is_open())
{
    while ( getline (infile,line))
    {
       testinp[i] = stod(line);
       i++; 
    }
}

infile.close() ;
 
 
 
    /* inputs order:
    fuel fraction, adiabatic flame temp. [K], pressure, ch4o2, h2air
    output:
    log10(cellSize [mm])
    Below is a test for H2-air, 30% of fuel,
     */
  
    double testout = 0.0;

    double *inp = testinp;
    double *out = &testout;

    export_eval(inp, out);

    *out = pow(10,*out);

    printf("%f", *out);

    return 0;
}

In [0]:
print(output)
import re
output_1 = str(output)
output_2 = re.findall('\d*\.?\d+',output_1)
output_3 = float(output_2[0])
output_4 = int(output_3)

outgeo = geo(output_4,n)

print(f"The range of optimum length: {outgeo[0]} < length < {outgeo[1]}, the range of outer diameter: {outgeo[2]} < outer diamemer < {outgeo[3]} .")

2.122307

The range of optimum length: 56 < length < 136, the range of outer diameter: 28 < outer diamemer < 68 .
